In [2]:
import pandas as pd
import json
import logging
from collections import Counter


# Create a custom logger
logger = logging.getLogger("@novo")
logger.setLevel(logging.INFO)

csv_file = 'DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_10.csv'
json_file = 'drug-ndc-0001-of-0001.json'


logger.info('Pipeline started....')


def json_as_dict(json_file):
    # Load the JSON data and create a dictionary to improve performance
    json_data_dict = {}
    with open(json_file, 'r') as json_file:
        json_data = json.load(json_file)['results']
        for product in json_data:
            json_data_dict[product['product_ndc']] = product
    return json_data_dict


def convert_to_10_digit_ndc(prod_srvc_id):
    # Convert prod_srvc_id to 11-digit NDC format
    prod_srvc_id = str(prod_srvc_id).zfill(11)

    # Add hyphens from right to left
    ndc = prod_srvc_id[:5] + '-' + prod_srvc_id[5:9] + '-' + prod_srvc_id[9:]

    # Split the NDC by hyphen
    parts = ndc.split('-')

    # Remove one leading zero from the labeler part
    parts[0] = parts[0][1:] if parts[0].startswith('0') else parts[0]

    # Remove one leading zero from the product part
    parts[1] = parts[1][1:] if parts[1].startswith('0') else parts[1]

    # Remove one leading zero from the package part
    parts[2] = parts[2][1:] if parts[2].startswith('0') else parts[2]
    
    # Return the 10-digit NDC in the format labeler-product
    return f"{parts[0]}-{parts[1]}-{parts[2]}"


def remove_package_part(ndc):
    parts = ndc.split('-')
    return f"{parts[0]}-{parts[1]}"


def create_csv_patients_with_insullin(csv_data, json_data_dict):
    """
    Create a CSV file with information about unique persons prescribed insulin.
    """
    # Create a list to store the linked data
    data_lst = []
    # To count unique persons ids
    unique_persons = set()
    
    # Iterate through the CSV data and process each row
    for index, row in csv_data.iterrows():
        prod_srvc_id = row['PROD_SRVC_ID']
        ndc = remove_package_part(convert_to_10_digit_ndc(prod_srvc_id))

        # Find the matching drug information in the JSON data dictionary
        if ndc in json_data_dict:
            product = json_data_dict[ndc]
            try:
                if "insulin" in product['active_ingredients'][0]['name'].lower():
                    unique_persons.add(row["DESYNPUF_ID"])
                    
                    data_dict = {
                        'DESYNPUF_ID': row['DESYNPUF_ID'],
                        'INSULIN': product['active_ingredients'][0]['name']
                    }
                    data_lst.append(data_dict)
            except KeyError as e:
                logger.warning(f"KeyError occurred Skipping row... Key: {str(e)} for product: {ndc}")
                continue
    data_lst.append({'unique_persons': len(unique_persons)})
    create_csv(data_lst, 'insulin')


def missing_packages(csv_data, json_data_dict):
    """
    Create a CSV file with information about missing packages.
    """
    # Create a list to store the data
    data_lst = []

    # Iterate through the CSV data and process each row
    for index, row in csv_data.iterrows():
        prod_srvc_id = row['PROD_SRVC_ID']
        ndc = remove_package_part(convert_to_10_digit_ndc(prod_srvc_id))

        # Find the matching drug information in the JSON data dictionary
        if ndc not in json_data_dict:
            try:
                data_dict = {
                    'PRODUCT_NDC': ndc,
                    'PACKAGE_NDC': convert_to_10_digit_ndc(prod_srvc_id)
                }
                data_lst.append(data_dict)
            except KeyError as e:
                logger.warning(f"KeyError occurred Skipping row... Key: {str(e)} for product: {ndc}")
                continue

    create_csv(data_lst, 'missing_packages')


def create_ai_exist_in_fda(csv_data, json_data_dict):
    """
    Create a CSV file with information about active ingredients found in FDA's database.
    """
    # Create a list to store the data
    data_lst = []
    # Create a counter to store the counts of active ingredients
    active_ingredient_counts = Counter()
    
    # Iterate through the CSV data and process each row
    for index, row in csv_data.iterrows():
        prod_srvc_id = row['PROD_SRVC_ID']
        ndc = remove_package_part(convert_to_10_digit_ndc(prod_srvc_id))

        # Find the matching drug information in the JSON data dictionary
        if ndc in json_data_dict:
            product = json_data_dict[ndc]
            try:
                active_ingredient = product['active_ingredients'][0]['name']
            except KeyError as e:
                logger.warning(f"KeyError occurred Skipping row... Key: {str(e)} for product: {ndc}")
                continue
                
            active_ingredient_counts[active_ingredient] += 1
    
    # Get the top tenactive ingredients and their corresponding person counts
    top_active_ingredients = active_ingredient_counts.most_common(10)

    for active_ingredient, person_count in top_active_ingredients:               
        data_dict = {
            'active_ingredient': active_ingredient,
            'person_count': person_count
        }
        data_lst.append(data_dict)

    create_csv(data_lst, 'ai_exist_in_fda')
    
        
def create_drug_not_found_in_fda(csv_data, json_data_dict):
    """
    Create a CSV file with information about drugs not found in FDA's database.
    """
    # Create a list to store the drugs not found in the FDA's database
    not_found_drugs = []
    # Create a list to store the data
    data_lst = []
    # Create a counter to store the prescription counts for drugs not found in the FDA's database
    prescription_counts = Counter()
    
    # Iterate through the CSV data and process each row
    for index, row in csv_data.iterrows():
        prod_srvc_id = row['PROD_SRVC_ID']
        ndc = remove_package_part(convert_to_10_digit_ndc(prod_srvc_id))

        # Check if the NDC is not found in the FDA's database
        if ndc not in json_data_dict:
            not_found_drugs.append(prod_srvc_id)
            prescription_counts[prod_srvc_id] += 1
                
    # Get the top ten drugs with the highest prescription counts
    top_drugs = prescription_counts.most_common(10)

    for drug, prescription_count in top_drugs:
        data_dict = {
            'drug': drug,
            'person_count': prescription_count
        }
        data_lst.append(data_dict)

    create_csv(data_lst, 'drugs_not_found_in_fda')


def create_csv(data, name):
    """
    Create a CSV file from the provided data.
    """
    # Create a DataFrame from the linked data
    data_df = pd.DataFrame(data)

    # Save the data to a CSV file
    output_file = f'{name}.csv'
    data_df.to_csv(output_file, index=False)

    print(f"Data saved to '{output_file}'.")


# Load the CSV data
csv_data = pd.read_csv(csv_file)

# Drop duplicate rows if they exist in the CSV file
csv_data = csv_data.drop_duplicates()

# Load the JSON data as a dictionary
json_data_dict = json_as_dict(json_file)

# Create CSV file with information about unique persons prescribed insulin
create_csv_patients_with_insullin(csv_data, json_data_dict)

# Create CSV file with information about missing packages
missing_packages(csv_data, json_data_dict)

# Create CSV file with information about active ingredients found in FDA's database
create_ai_exist_in_fda(csv_data, json_data_dict)

# Create CSV file with information about drugs not found in FDA's database
create_drug_not_found_in_fda(csv_data, json_data_dict)


KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0024-5824
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 66887-003
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0026-3782
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58394-013
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-309
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0024-5151
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-306
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49502-416
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 37000-514
KeyE

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 66887-003
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-8147
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 67402-021
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9971
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 24208-539
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0026-3782
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 51316-106
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-3663
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-306
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 24385-451
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58394-012
Key

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9971
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9963
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 67402-021
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7635
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 24208-539
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-3663
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 55566-1501
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58394-013
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0378-8688
K

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0169-7065
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-4339
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-3641
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-306
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 42546-270
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0013-2654
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 24208-539
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 11704-777
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7635
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58406-425
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9974
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 59088-756
Key

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-306
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 11704-777
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 24208-539
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-2108
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-308
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9973
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9964
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9971
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-309
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 51316-106
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 11704-620
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65628-063
K

...

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 11704-777
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58406-425
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58406-425
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 44087-3388
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58394-012
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0169-7065
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-308
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0026-3783
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9972
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 37000-033
Ke

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-8031
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7637
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0781-7054
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0026-3782
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65628-063
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-8031
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49502-416
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65044-9964
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7636
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
Key

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-307
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58394-012
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 11704-620
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 67457-195
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 50458-308
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 58406-425
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 67402-021
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 59088-756
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7635
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 49884-944
KeyE

KeyError occurred Skipping row... Key: 'active_ingredients' for product: 37000-033
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 55566-1501
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7635
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 24208-539
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-3346
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0026-3786
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0074-3641
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0026-3782
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-8031
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 0002-7637
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 13533-700
KeyError occurred Skipping row... Key: 'active_ingredients' for product: 65649-201
Key

Data saved to 'ai_exist_in_fda.csv'.
Data saved to 'drugs_not_found_in_fda.csv'.
